In [3]:
import re
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F

In [5]:
data = pd.read_csv("data/train.csv")
data1 = pd.read_csv("data/test.csv")

In [7]:
df=data+data1

In [10]:
data

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."
...,...,...,...
119995,1,Pakistan's Musharraf Says Won't Quit as Army C...,KARACHI (Reuters) - Pakistani President Perve...
119996,2,Renteria signing a top-shelf deal,Red Sox general manager Theo Epstein acknowled...
119997,2,Saban not going to Dolphins yet,The Miami Dolphins will put their courtship of...
119998,2,Today's NFL games,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...


In [ ]:
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphanumeric characters
    text = text.lower()  # Convert to lowercase
    return text

data['cleaned_text'] = data['text'].apply(clean_text)

# Step 2: Tokenize and build vocabulary
def yield_tokens(data_iter):
    for text in data_iter:
        yield text.split()

vocab = build_vocab_from_iterator(yield_tokens(data['cleaned_text']), specials=["<pad>", "<unk>"])
vocab.set_default_index(vocab["<unk>"])  # Set default index for unknown words

# Step 3: Convert text to sequences
data['text_sequences'] = data['cleaned_text'].apply(lambda x: [vocab[token] for token in x.split()])

# Step 4: Pad sequences to uniform length
MAX_SEQ_LEN = 100

def pad_sequence(seq, max_length):
    return seq[:max_length] + [vocab["<pad>"]] * max(0, max_length - len(seq))

data['padded_sequences'] = data['text_sequences'].apply(lambda x: pad_sequence(x, MAX_SEQ_LEN))

# Step 5: Encode labels
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['category'])

In [ ]:
class AGNewsDataset(Dataset):
    def __init__(self, data):
        self.texts = torch.tensor(data['padded_sequences'].tolist(), dtype=torch.long)
        self.labels = torch.tensor(data['label'].values, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

# Split into train and test sets
train_data, test_data = train_test_split(data, test_size=0.4, random_state=42)
train_dataset = AGNewsDataset(train_data)
test_dataset = AGNewsDataset(test_data)

# Create DataLoaders
torch.manual_seed(42)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.fc1 = nn.Linear(embed_dim * MAX_SEQ_LEN, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.embedding(x)  # [batch_size, seq_len, embed_dim]
        x = x.view(x.size(0), -1)  # Flatten to [batch_size, embed_dim * seq_len]
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Initialize the model
vocab_size = len(vocab)
embed_dim = 100
num_classes = len(label_encoder.classes_)
model = TextClassificationModel(vocab_size, embed_dim, num_classes)